<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Clustering-based" data-toc-modified-id="Clustering-based-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Clustering based</a></span><ul class="toc-item"><li><span><a href="#modeling" data-toc-modified-id="modeling-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>modeling</a></span></li></ul></li><li><span><a href="#LDA-based" data-toc-modified-id="LDA-based-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>LDA based</a></span><ul class="toc-item"><li><span><a href="#tag-parser" data-toc-modified-id="tag-parser-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>tag parser</a></span></li><li><span><a href="#Kmeans-clustering-based-on-LDA-topic-distribution-representation" data-toc-modified-id="Kmeans-clustering-based-on-LDA-topic-distribution-representation-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Kmeans clustering based on LDA topic distribution representation</a></span></li></ul></li></ul></div>

In [2]:
seeds = 123

In [3]:
train = pd.read_json('../data/structured_train.json')
test = pd.read_json('../data/structured_test.json')

In [4]:
# train = train.groupby('label').sample(50, random_state=seeds)
# test = test.groupby('label').sample(50, random_state=seeds)

In [5]:
select_cols = ["global_index", "doc_path", "label", 
               "reply", "reference_one", "reference_two","tag_reply", "tag_reference_one", "tag_reference_two", 
               "Subject", "From", "Lines", "Organization", "contained_emails", "long_string", "text", "error_message"
               ]
print("\nmay use cols: \n", select_cols)
train = train[select_cols]
test = test[select_cols]


may use cols: 
 ['global_index', 'doc_path', 'label', 'reply', 'reference_one', 'reference_two', 'tag_reply', 'tag_reference_one', 'tag_reference_two', 'Subject', 'From', 'Lines', 'Organization', 'contained_emails', 'long_string', 'text', 'error_message']


# Clustering based
- Steps:
    1. Transform into TF-IDF matrix
    2. Dimension reduction into 200
    3. Clustering in cosine similarity space (since it is word)
    4. Assign labels with majority vote based on training set labels
    5. Prediction
        1. Transform test set into TF-IDF matrix
        2. Dimension reduction into 200
        3. Make prediction based on the clusters and mapping between clusters and labels from training set
    6. Evaluation
        1. Based on classification report

## modeling

# LDA based 

## tag parser

In [6]:
from clustering_utils import *
# count_vectorizer, dimension_reduction


In [7]:
num_topics = 100

# train_text = train['tag_reply']
# train_text = train['tag_reply']+ ' ' + train['tag_reference_one']
train_text = train['reply'] + ' ' + train['reference_one']
train_label = train['label']


# test_text  = test['tag_reply'] 
# test_text  = test['tag_reply'] + ' ' + test['tag_reference_one']
test_text  = test['reply'] + ' ' + test['reference_one']
test_label = test['label']

lda, voc = fit_topic_model(train_text, num_topics=num_topics, save_name='gensim_lda_model')
# lda = load_gensim_LDA_model(save_name='lda_gensim_model')

In [9]:
# vis_lda = visualize_LDA_model(train_text, voc, lda)
# vis_lda

In [11]:
train_pred_group, train_topic_distribution = pred_topic_model(lda, train_text, vocabulary=voc)
group_to_label = link_group_to_label(train_label, train_pred_group)

test_pred_group, test_topic_distribution = pred_topic_model(lda, test_text, vocabulary=voc)
test_pred = test_pred_group.apply(lambda group: group_to_label[group])

Series([], dtype: float64)
Group to label mapping: 
Group 0 <-> label comp.os.ms-windows.misc
Group 1 <-> label comp.sys.ibm.pc.hardware
Group 2 <-> label no_group
Group 3 <-> label comp.sys.ibm.pc.hardware
Group 4 <-> label comp.graphics
Group 5 <-> label comp.windows.x
Group 6 <-> label no_group
Group 7 <-> label alt.atheism
Group 8 <-> label comp.graphics
Group 9 <-> label no_group
Group 10 <-> label no_group
Group 11 <-> label no_group
Group 12 <-> label comp.sys.ibm.pc.hardware
Group 13 <-> label talk.politics.guns
Group 14 <-> label no_group
Group 15 <-> label no_group
Group 16 <-> label alt.atheism
Group 17 <-> label comp.sys.ibm.pc.hardware
Group 18 <-> label comp.sys.ibm.pc.hardware
Group 19 <-> label comp.graphics
Group 20 <-> label comp.graphics
Group 21 <-> label alt.atheism
Group 22 <-> label no_group
Group 23 <-> label misc.forsale
Group 24 <-> label alt.atheism
Group 25 <-> label comp.graphics
Group 26 <-> label no_group
Group 27 <-> label alt.atheism
Group 28 <-> label 

In [12]:
print(metrics.classification_report(y_true = test_label, y_pred=test_pred))

                          precision    recall  f1-score   support

             alt.atheism       0.15      0.08      0.10       319
           comp.graphics       0.23      0.15      0.18       389
 comp.os.ms-windows.misc       0.12      0.02      0.04       394
comp.sys.ibm.pc.hardware       0.20      0.65      0.31       392
   comp.sys.mac.hardware       0.00      0.00      0.00       385
          comp.windows.x       0.28      0.23      0.25       395
            misc.forsale       0.32      0.08      0.12       390
                no_group       0.00      0.00      0.00         0
               rec.autos       0.08      0.03      0.04       395
         rec.motorcycles       0.08      0.08      0.08       398
      rec.sport.baseball       0.11      0.12      0.11       397
        rec.sport.hockey       1.00      0.00      0.00       827
               sci.crypt       0.00      0.00      0.00       396
         sci.electronics       0.00      0.00      0.00       393
         

C:\Users\Administrator\Anaconda3\envs\py810\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\Anaconda3\envs\py810\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\Anaconda3\envs\py810\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Use

## Kmeans clustering based on LDA topic distribution representation

In [43]:

def fit_clustering_model(dtm_train, train_label, num_clusters, metric='Cosine', model='KMeans', repeats=20):
    '''

    '''
    assert metric in ['Cosine', 'L2']
    assert model in ['KMeans']

    # model training
    if model == 'KMeans':
        if metric == 'Cosine':
            # normalise has to be False!
            # clusterer = KMeansClusterer(num_clusters, cosine_distance, normalise=False, repeats=repeats, avoid_empty_clusters=True)
            # clusterer = KMeansClusterer(num_clusters, cosine, normalise=False, repeats=repeats, avoid_empty_clusters=True)
            clusterer = KMeansClusterer(num_clusters, cosine, normalise=False, repeats=repeats, avoid_empty_clusters=False)
            train_cluster_pred = clusterer.cluster(dtm_train, assign_clusters=True)
        elif metric == 'L2':
            clusterer = KMeans(n_clusters=num_clusters, n_init=repeats).fit(dtm_train)
            train_cluster_pred = clusterer.labels_.tolist()

    elif model == 'GMM':
        pass
        # GMM model not good in such case
        # clusterer = mixture.GaussianMixture(n_components=num_clusters, n_init=repeats, covariance_type='diag')
        # clusterer.fit(dtm_train)
        # train_cluster_pred = clusterer.predict(dtm_train)

    # Maping clusters into labels
    clusters_to_labels = link_group_to_label(train_label, train_cluster_pred, num_clusters)
    return clusterer, clusters_to_labels

In [45]:
dtm_train, transform_mapper = dimension_reduction(train_topic_distribution, out_dim=2)
dtm_test = transform_mapper.transform(test_topic_distribution)

Dimension reduction with truncate SVD:
   input columns with  100
   output columns with  2


In [47]:
clusterer, clusters_to_labels = fit_clustering_model(dtm_train, train_label, num_clusters=10, metric='Cosine', repeats=1)
pred = pred_clustering_model(dtm_test, clusterer, clusters_to_labels)
print(metrics.classification_report(y_true = test_label, y_pred=pred))

Group to label mapping: 
Group 0 <-> label comp.sys.ibm.pc.hardware
Group 1 <-> label comp.sys.mac.hardware
Group 2 <-> label rec.autos
Group 3 <-> label sci.med
Group 4 <-> label comp.sys.mac.hardware
Group 5 <-> label rec.motorcycles
Group 6 <-> label comp.windows.x
Group 7 <-> label rec.autos
Group 8 <-> label soc.religion.christian
Group 9 <-> label comp.windows.x


                          precision    recall  f1-score   support

             alt.atheism       0.00      0.00      0.00       319
           comp.graphics       0.00      0.00      0.00       389
 comp.os.ms-windows.misc       0.00      0.00      0.00       394
comp.sys.ibm.pc.hardware       0.18      0.27      0.22       392
   comp.sys.mac.hardware       0.11      0.27      0.16       385
          comp.windows.x       0.11      0.22      0.14       395
            misc.forsale       0.00      0.00      0.00       390
               rec.autos       0.09      0.25      0.13       395
         rec.motorcycles       0

C:\Users\Administrator\Anaconda3\envs\py810\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\Anaconda3\envs\py810\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\Anaconda3\envs\py810\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [ ]:
KMeansClusterer()

In [39]:
from scipy.spatial.distance import cosine
a = np.array([0,0])
b = np.array([0,0])
cosine(a,b)

C:\Users\Administrator\Anaconda3\envs\py810\lib\site-packages\scipy\spatial\distance.py:699: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


0

In [ ]:
KMeans()

In [33]:
cosine_distance(0,0)

C:\Users\Administrator\Anaconda3\envs\py810\lib\site-packages\nltk\cluster\util.py:131: RuntimeWarning: invalid value encountered in true_divide
  return 1 - (numpy.dot(u, v) / (sqrt(numpy.dot(u, u)) * sqrt(numpy.dot(v, v))))


nan